In [1]:
import numpy as np


def complementary_channel(kraus_ops: list[np.ndarray]) -> list[np.ndarray]:
    r"""Compute the Kraus operators for the complementary map of a quantum channel.

    (Section: Representations and Characterizations of Channels from :cite:`Watrous_2018_TQI`).

    The complementary map is derived from the given quantum channel's Kraus operators by
    rearranging the rows of the input Kraus operators into the Kraus operators of the
    complementary map.

    Specifically, for each Kraus operator :math:`K_i` in the input channel :math:`\Phi`,
    we define the complementary Kraus operators :math:`K_i^C` by stacking the rows of
    :math:`K_i` from all Kraus operators vertically.

    References
    ==========
    .. bibliography::
        :filter: docname in docnames

    :raises ValueError: If the input is not a valid list of Kraus operators.
    :param kraus_ops: A list of numpy arrays representing the Kraus operators of a quantum channel.
                      Each Kraus operator is assumed to be a square matrix.
    :return: A list of numpy arrays representing the Kraus operators of the complementary map.

    """
    num_kraus = len(kraus_ops)
    print("hello: ")
    print(num_kraus)
    print("hello2")
    if num_kraus==0:
        raise ValueError("All Kraus operators must be non-empty matrices.")
    op_dim = kraus_ops[0].shape[0]

    if any(k.shape[0] != k.shape[1] for k in kraus_ops):
        raise ValueError("All Kraus operators must be square matrices.")

    if any(k.shape[0] != op_dim for k in kraus_ops):
        raise ValueError("All Kraus operators must be equal size matrices.")
    
    # Check the Kraus completeness relation: ∑ K_i† K_i = I
    identity = np.eye(op_dim, dtype=kraus_ops[0].dtype)
    # print("printing products: ")
    # for k in kraus_ops:
    #     print(k.T.conj() @ k)

    sum_k_dagger_k = sum(k.T.conj() @ k for k in kraus_ops)

    print("sumk: ",sum_k_dagger_k)
    
    if not np.allclose(sum_k_dagger_k, identity):
        raise ValueError("The Kraus operators do not satisfy the completeness relation ∑ K_i† K_i = I.")

    comp_kraus_ops = []

    for row in range(op_dim):
        comp_kraus_op = np.vstack([kraus_ops[i][row, :] for i in range(num_kraus)])
        comp_kraus_ops.append(comp_kraus_op)

    return comp_kraus_ops


In [2]:
# Define test cases for complementary map
kraus_1 = np.array([[1, 0], [0, 0]]) / np.sqrt(2)
kraus_2 = np.array([[0, 1], [0, 0]]) / np.sqrt(2)
kraus_3 = np.array([[0, 0], [1, 0]]) / np.sqrt(2)
kraus_4 = np.array([[0, 0], [0, 1]]) / np.sqrt(2)

# Expected results for the complementary map
expected_res_comp = [
    np.array([[1, 0], [0, 1], [0, 0], [0, 0]]) / np.sqrt(2),
    np.array([[0, 0], [0, 0], [1, 0], [0, 1]]) / np.sqrt(2),
]

# Higher-dimensional Kraus operators (3x3)
kraus_5 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) / np.sqrt(3)
kraus_6 = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]) / np.sqrt(3)
kraus_7 = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]]) / np.sqrt(3)

expected_res_comp_high_dim = [
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]) / np.sqrt(3),
    np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]) / np.sqrt(3),
    np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]]) / np.sqrt(3),
]

# Single Kraus operator (edge case)
kraus_single = np.array([[1, 0], [0, 1]])

expected_res_single =[
    np.array([[1, 0]]),
    np.array([[0, 1]]),
] 


In [3]:
complementary_channel([kraus_5, kraus_6, kraus_7])

hello: 
3
hello2
sumk:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


[array([[0.57735027, 0.        , 0.        ],
        [0.        , 0.57735027, 0.        ],
        [0.        , 0.        , 0.57735027]]),
 array([[0.        , 0.57735027, 0.        ],
        [0.        , 0.        , 0.57735027],
        [0.57735027, 0.        , 0.        ]]),
 array([[0.        , 0.        , 0.57735027],
        [0.57735027, 0.        , 0.        ],
        [0.        , 0.57735027, 0.        ]])]

In [4]:
import pytest
@pytest.mark.parametrize(
    "kraus_ops, expected",
    [
        # Test complementary_channel on a set of 2x2 Kraus operators (the ones you gave).
        ([kraus_1, kraus_2, kraus_3, kraus_4], expected_res_comp),
        # Test complementary_channel with higher-dimensional (3x3) Kraus operators.
        ([kraus_5, kraus_6], expected_res_comp_high_dim),
        # Test complementary_channel with a single Kraus operator (edge case).
        ([kraus_single], expected_res_single),
    ],
)
def test_complementary_channel(kraus_ops, expected):
    """Test complementary_channel works as expected for valid inputs."""
    calculated = complementary_channel(kraus_ops)

    # Compare the shapes first to debug broadcasting issues
    assert len(calculated) == len(expected), "Mismatch in number of Kraus operators"
    for calc_op, exp_op in zip(calculated, expected):
        assert np.isclose(calc_op, exp_op, atol=1e-6).all()

In [5]:
test_complementary_channel([kraus_single], expected_res_single)

hello: 
1
hello2
sumk:  [[1 0]
 [0 1]]


In [6]:
complementary_channel([kraus_single])

hello: 
1
hello2
sumk:  [[1 0]
 [0 1]]


[array([[1, 0]]), array([[0, 1]])]

In [7]:
expected_res_single

[array([[1, 0]]), array([[0, 1]])]

In [8]:
@pytest.mark.parametrize(
    "kraus_ops",
    [
        # Invalid test case: non-square matrices
        ([np.array([[1, 0, 0], [0, 1, 0]])]),  # Not a square matrix
        # Invalid test case: empty list of Kraus operators
        ([]),
        # Invalid test case: single row matrix (not a square)
        ([np.array([[1, 0]])]),
        # Different dimenisions for kraus operators in a set
        ([np.array([[1, 0, 0], [0, 1, 0], [0, 1, 1]]), np.array([[1, 0], [0, 1]])]),
        # Invalid test case: Kraus operators that do not satisfy the completeness relation
        ([np.array([[1, 0], [0, 0.5]]), np.array([[0, 0.5], [0, 0.5]])]),  # Sum != I
    ],
)
def test_complementary_channel_error(kraus_ops):
    """Test function raises error as expected for invalid inputs."""
    with pytest.raises(ValueError):
        complementary_channel(kraus_ops)


In [13]:
complementary_channel([np.array([[1, 0, 0], [0, 1, 0]])])

hello: 
1
hello2


ValueError: All Kraus operators must be square matrices.